<a href="https://colab.research.google.com/github/ratminurisnaini/Customer-Churn-Prediction/blob/main/Customer_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, classification_report

# Data Understanding

In [ ]:
# Read Data
df = pd.read_csv('/content/telco-churn.csv')

# Check the data, num of rows, and num of columns
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [ ]:
# Check features and label for machine learning model
print('Columns for Features')
print(df.iloc[:,:-1].columns)
print('\nColumn for Label')
print(df.iloc[:,-1:].columns)

Columns for Features
Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges'],
      dtype='object')

Column for Label
Index(['Churn'], dtype='object')


In [ ]:
# Check the data types of all columns
for col in df.columns:
    print(col, ':', df[col].dtypes)
    print(df[col].unique(), '\n')

customerID : object
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK'] 

gender : object
['Female' 'Male'] 

SeniorCitizen : int64
[0 1] 

Partner : object
['Yes' 'No'] 

Dependents : object
['No' 'Yes'] 

tenure : int64
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39] 

PhoneService : object
['No' 'Yes'] 

MultipleLines : object
['No phone service' 'No' 'Yes'] 

InternetService : object
['DSL' 'Fiber optic' 'No'] 

OnlineSecurity : object
['No' 'Yes' 'No internet service'] 

OnlineBackup : object
['Yes' 'No' 'No internet service'] 

DeviceProtection : object
['No' 'Yes' 'No internet service'] 

TechSupport : object
['No' 'Yes' 'No internet service'] 

StreamingTV : object
['No' 'Yes' 'No internet service'] 

StreamingMovies : object
['No' 'Yes' 'No internet service'] 

Contract :

From the previous cell output, the SeniorCitizen and TotalCharges columns have incorrect data types. The data type for SeniorCitizen should be string with value 'No' for 0 and 'Yes' for 1, and the data type for TotalCharges should be float.

So we will replace the values in SeniorCitizen column and change the data type in TotalCharges column.

In [ ]:
# Replace the values and change the data type to string in SeniorCitizen column
df['SeniorCitizen'].replace({0:'No', 1:'Yes'}, inplace=True)

# Change TotalCharges data type to float
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [ ]:
# Recheck the data types of all columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   object 
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
# Explore each columns
for col in df.columns:
    print('-'*len(col) + '\n' + col + '\n' + '-'*len(col))
    print('DESCRIPTIVE STATISTICS')
    print(df[col].describe(), '\n')
    print('VALUE COUNTS')
    print(df[col].value_counts(), '\n')

----------
customerID
----------
DESCRIPTIVE STATISTICS
count           7043
unique          7043
top       7590-VHVEG
freq               1
Name: customerID, dtype: object 

VALUE COUNTS
customerID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64 

------
gender
------
DESCRIPTIVE STATISTICS
count     7043
unique       2
top       Male
freq      3555
Name: gender, dtype: object 

VALUE COUNTS
gender
Male      3555
Female    3488
Name: count, dtype: int64 

-------------
SeniorCitizen
-------------
DESCRIPTIVE STATISTICS
count     7043
unique       2
top         No
freq      5901
Name: SeniorCitizen, dtype: object 

VALUE COUNTS
SeniorCitizen
No     5901
Yes    1142
Name: count, dtype: int64 

-------
Partner
-------
DESCRIPTIVE STATISTICS
count     7043
unique       2
top         No
freq      3641
Name: Partner, dtype: ob

**Check the interesting and odd things in the data**

Check 'tenure' column

In [ ]:
# The interesting things:
# 1. The number of customer with the longest subcription time is in the second rank
# 2. The number of customer with the shortest subcription time is in the last rank
print('The longest subcription time: ', df['tenure'].max())
print('The shortest subcription time: ', df['tenure'].min())
pd.DataFrame({'Tenure':df['tenure'].value_counts().index,
              'Count':df['tenure'].value_counts().values})

The longest subcription time:  72
The shortest subcription time:  0


,Tenure,Count
0,1,613
1,72,362
2,2,238
3,3,200
4,4,176
...,...,...
68,28,57
69,39,56
70,44,51
71,36,50


In [ ]:
# 3. All customers who newly subscribed (0 months subscription) are staying (no churn)
df[df['tenure'] == 0]['Churn'].value_counts()

,count
Churn,
No,11


In [ ]:
# 4. Customers who have subscribed for 1 month, have a higher churn rate
df[df['tenure'] == 1]['Churn'].value_counts()

,count
Churn,
Yes,380
No,233


In [ ]:
# 5. Customers with the longest subscriptions, more are staying
df[df['tenure'] == 72]['Churn'].value_counts()

,count
Churn,
No,356
Yes,6


Check 'PhoneService' and 'MultipleLines' columns

In [ ]:
# Nothing odd
# In MultipleLines column, customers who have 'No phone service' also have no phone service in PhoneService column
df[['PhoneService', 'MultipleLines']].value_counts()

PhoneService  MultipleLines   
Yes           No                  3390
              Yes                 2971
No            No phone service     682
Name: count, dtype: int64

Check 'InternetService' until 'StreamingMovies' columns

In [ ]:
# Nothing odd
# Customers who have no internet sevice also have no internet service in OnlineSecurity until StreamingMovies columns
df[list(df.loc[:, 'InternetService' : 'StreamingMovies'].columns)].value_counts()

InternetService  OnlineSecurity       OnlineBackup         DeviceProtection     TechSupport          StreamingTV          StreamingMovies    
No               No internet service  No internet service  No internet service  No internet service  No internet service  No internet service    1526
Fiber optic      No                   No                   No                   No                   No                   No                      399
DSL              No                   No                   No                   No                   No                   No                      294
Fiber optic      No                   No                   No                   No                   Yes                  Yes                     181
                                      Yes                  Yes                  No                   Yes                  Yes                     164
                                                                                                                                                 ... 
                 Yes                  No                   Yes                  No                   No                   Yes                       9
                                                                                Yes                  No                   No                        9
                                                           No                   Yes                  No                   Yes                       8
                                                                                                     Yes                  No                        7
                                                           Yes                  Yes                  Yes                  No                        3
Name: count, Length: 129, dtype: int64

Check 'PhoneService' and 'InternetService' columns

In [ ]:
# Nothing odd
# Customers who have subscribed certainly have phone service and/or internet service
df[['PhoneService', 'InternetService']].value_counts()

PhoneService  InternetService
Yes           Fiber optic        3096
              DSL                1739
              No                 1526
No            DSL                 682
Name: count, dtype: int64

Check 'tenure', 'MonthlyCharges', and 'TotalCharges' columns

In [ ]:
# The interesting thing:
# Mostly customer's 'TotalCharges' don't seemed to be calculated based on 'tenure' times 'MonthlyCharges'.
# Maybe there are conditions such as discounts or arrears that make 'TotalCharges' not equal to 'tenure' times 'MonthlyCharges'.
print('Num of data where "TotalCharges == tenure * MonthlyCharges":', (df['TotalCharges'] == df['tenure'] * df['MonthlyCharges']).sum())
print('Num of data where "TotalCharges != tenure * MonthlyCharges":', (df['TotalCharges'] != df['tenure'] * df['MonthlyCharges']).sum())

Num of data where "TotalCharges == tenure * MonthlyCharges": 614
Num of data where "TotalCharges != tenure * MonthlyCharges": 6429


In [ ]:
# Check which subscription times that have all the TotalCharges calculated based on tenure times MonthlyCharges
a = df[df['TotalCharges'] == df['tenure'] * df['MonthlyCharges']]['tenure'].unique()
b = df[df['TotalCharges'] != df['tenure'] * df['MonthlyCharges']]['tenure'].unique()

print('Subscription time (month\s):', set(a).difference(set(b)))

Subscription time (month\s): {1}


From the previous results, it turns out that all customers who have subscription time for 1 month have 'TotalCharges' calculated based on the 'tenure' times 'MonthlyCharges'.

Check the churned customer percentage in the data

In [ ]:
# Churn rate
df['Churn'].value_counts(normalize=True).apply(lambda x: str(round(x*100, 2)) + '%')

,proportion
Churn,
No,73.46%
Yes,26.54%


# Data Cleaning

Check missing values in the data

In [ ]:
# There are missing values in the 'TotalCharges' column because of the previous process when we changed the data type from string to float
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [ ]:
# The null values in TotalCharges column occur when the tenure values are 0 month
idx = df[df['TotalCharges'].isnull()].index
df.loc[idx, ['tenure', 'TotalCharges']]

,tenure,TotalCharges
488,0,NaN
753,0,NaN
936,0,NaN
1082,0,NaN
1340,0,NaN
3331,0,NaN
3826,0,NaN
4380,0,NaN
5218,0,NaN
6670,0,NaN


In [ ]:
# We will impute the null values with 0 because we assume that the customers don't have TotalCharges yet in 0 month
df['TotalCharges'].fillna(0, inplace=True)
df.loc[idx, ['tenure', 'TotalCharges']]

,tenure,TotalCharges
488,0,0.0
753,0,0.0
936,0,0.0
1082,0,0.0
1340,0,0.0
3331,0,0.0
3826,0,0.0
4380,0,0.0
5218,0,0.0
6670,0,0.0


Check duplicate data

In [ ]:
# Drop ID column first to check duplicate values
df.drop('customerID', axis=1, inplace=True)

In [ ]:
# Check num of duplicates
df.duplicated().sum()

22

In [ ]:
# Check data shape before removing duplicate values
print('Data shape before removing duplicate values:', df.shape)

# Remove the duplicate values
df.drop_duplicates(inplace=True)

# Check data shape after removing duplicate values
print('Data shape after removing duplicate values', df.shape)

Data shape before removing duplicate values: (7043, 20)
Data shape after removing duplicate values (7021, 20)


# Exploratory Data Analysis

**Add new categorical column from 'tenure' column**

In [ ]:
bins = [0, 13, 25, 37, 49, 61, 73]
labels = ['0-1 year', '1-2 year', '2-3 year', '3-4 year', '4-5 year', '5-6 year']
df['TenureYear'] = pd.cut(df['tenure'], bins=bins, labels=labels, right=False)

**Create some graph functions**

In [ ]:
# Create color list for graph function
color_list_default=px.colors.qualitative.Plotly
color_list = color_list_default.copy()
color_list[0], color_list[1] = color_list[1], color_list[0]

In [ ]:
# Bar graph function for categorical data
def create_bar(col_name: str):
    if df[col_name].nunique() > 2:
        clr_list = color_list_default.copy()
    else:
        clr_list = color_list.copy()

    df_bar = df.sort_values(by=col_name, ascending=True)
    fig = px.histogram(x = df_bar['Churn'], color = df_bar[col_name], text_auto=True,
                       color_discrete_sequence=clr_list,
                       title='Churn vs ' + col_name)

    fig.update_layout(barmode = 'group', legend_title_text = col_name)
    fig.update_xaxes(title='Churn')
    fig.update_yaxes(title='Jumlah')
    fig.update_layout(width=500, height=400)

    return fig

In [ ]:
# Pie chart function for categorical data
def create_pie(col_name: str):
    if df[col_name].nunique() > 2:
        clr_list = color_list_default.copy()
    else:
        clr_list = color_list.copy()

    fig = make_subplots(rows = 2, cols = 2,
                        specs = [[{'type' : 'domain', 'rowspan' : 2}, {'type' : 'domain'}],
                                 [None                              , {'type' : 'domain'}]],
                        subplot_titles=['Total Customer', 'No Churn', 'Churn'],
                        vertical_spacing=0.1)

    fig.add_trace(go.Pie(labels = df[col_name].value_counts().index,
                         values = df[col_name].value_counts().values,
                         hole = 0.3,
                         pull = [0.01, 0.01, 0.01]),
                  row = [1,2], col = 1)
    fig.add_trace(go.Pie(labels = df[df.Churn == 'No'][col_name].value_counts().sort_index(ascending=True).index,
                         values = df[df.Churn == 'No'][col_name].value_counts().sort_index(ascending=True).values,
                         hole = 0.3,
                         pull = [0.01, 0.01, 0.01],
                         marker_colors=clr_list[:df[col_name].nunique()]),
                  row = 1, col = 2)
    fig.add_trace(go.Pie(labels = df[df.Churn == 'Yes'][col_name].value_counts().sort_index(ascending=True).index,
                         values = df[df.Churn == 'Yes'][col_name].value_counts().sort_index(ascending=True).values,
                         hole = 0.3,
                         pull = [0.01, 0.01, 0.01],
                         marker_colors=clr_list[:df[col_name].nunique()]),
                  row = 2, col = 2)

    fig.update_traces(textposition = 'inside', textinfo = 'percent+label')

    fig.update_layout(height = 600, title='Churn vs ' + col_name, legend_title_text=col_name)
    return fig

In [ ]:
# Histogram function for numerical data
def create_hist(col_name: str):
    fig = px.histogram(x = df[col_name], color = df['Churn'], opacity=0.6, nbins=100)
    fig.update_layout(barmode = 'overlay', title=col_name +' vs Churn', legend_title_text = 'Churn?')
    fig.update_xaxes(title = col_name)

    return fig

**EDA for 'Gender' column**

In [ ]:
create_bar('gender')

In [ ]:
create_pie('gender')

The proportion between male and female customers are almost equal in 'churn' and 'no churn' category.

**EDA for 'SeniorCitizen' column**

In [ ]:
create_bar('SeniorCitizen')

In [ ]:
create_pie('SeniorCitizen')

Most of the customers who use the provider are at the productive age. Because usually they need the phone or internet service to support their activities.

If we look at the bar graph, the number of senior citizens who churn is almost a half of the total number of senior citizens. Maybe because their children or families have subscribed another provider or they don't need it again.

**EDA for 'Partner' column**

In [ ]:
create_bar('Partner')

In [ ]:
create_pie('Partner')

The customers who churn mostly don't have a partner because they can decide to churn without feeling burdened.

**EDA for 'Dependents' column**

In [ ]:
create_bar('Dependents')

In [ ]:
create_pie('Dependents')

The visualizations of 'Dependents' column have similar patterns like 'Partner' column. The customers who don't have dependents tend to churn because they don't need to consider their dependents.

**EDA for 'Tenure' column**

In [ ]:
# Tenure column
create_hist('tenure')

In [ ]:
# TenureYear column
fig = create_bar('TenureYear')
fig.update_layout(width=500, height=500)

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1971: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
# TenureYear column
create_pie('TenureYear')

Customers who have stayed with the company for 5-6 years tend to not churn because they already believe in the service they are using.

New customers who have subscribed for less than 1 year tend to churn because they are in the adaptation period of using the service.

**EDA for 'PhoneService' column**

In [ ]:
create_bar('PhoneService')

In [ ]:
create_pie('PhoneService')

Most of the customers have PhoneService with their subscription.

**EDA for 'MultipleLines' column**

In [ ]:
create_bar('MultipleLines')

In [ ]:
create_pie('MultipleLines')

The comparison between customers who have multiple lines or not is almost equal.

**EDA for 'InternetService' column**

In [ ]:
create_bar('InternetService')

In [ ]:
create_pie('InternetService')

Most of customers who churn have internet service with fiber optic. This because fiber optic has a higher cost, so the customers tend to churn.

**EDA for 'OnlineSecurity' column**

In [ ]:
create_bar('OnlineSecurity')

In [ ]:
create_pie('OnlineSecurity')

Most of the customers who churn don't have online security. This could occur because they may have a bad experience with the security so they prefer to churn.

**EDA for 'OnlineBackup' column**

In [ ]:
create_bar('OnlineBackup')

In [ ]:
create_pie('OnlineBackup')

**EDA for 'DeviceProtection' column**

In [ ]:
create_bar('DeviceProtection')

In [ ]:
create_pie('DeviceProtection')

**EDA for 'TechSupport' column**

In [ ]:
create_bar('TechSupport')

In [ ]:
create_pie('TechSupport')

The visualizations of 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', and 'TechSupport' have similar patterns like 'OnlineSecurity' column where customers who churn mostly don't have that service because they may have bad experience with that.

**EDA for 'StreamingTV' column**

In [ ]:
create_bar('StreamingTV')

In [ ]:
create_pie('StreamingTV')

The proportion of customers who have internet service with streaming TV or not is almost equal.

**EDA for 'StreamingMovies' column**

In [ ]:
create_bar('StreamingMovies')

In [ ]:
create_pie('StreamingMovies')

The proportion of customers who have internet service with streaming movies or not is almost equal.

**EDA for 'Contract' column**

In [ ]:
create_bar('Contract')

In [ ]:
create_pie('Contract')

Most customers who churn have month-to-month contract terms. Customers with an annual subscription tend not to churn because they have already paid for the remaining months of their subscription.

**EDA for 'PaperlessBilling' column**

In [ ]:
create_bar('PaperlessBilling')

In [ ]:
create_pie('PaperlessBilling')

Customers who churn mostly chose paperless billing in their payment.

**EDA for 'PaymentMethod' column**

In [ ]:
create_bar('PaymentMethod')

In [ ]:
create_pie('PaymentMethod')

Customers who churn mostly chose electronic check in their payment method.

**EDA for 'MonthlyCharges' column**

In [ ]:
create_hist('MonthlyCharges')

Customers with low monthly cost are more likely to stay.

Most customers who churn have a monthly cost between 68-108 dollars. This could be due to the customers' economic conditions. They need to consider their ability to pay.

**EDA for 'TotalCharges' column**

In [ ]:
create_hist('TotalCharges')

Mostly customers who churn are customers with a total charges between 50-150 dollars.

**EDA for 'Churn' column**

In [ ]:
fig = px.pie(values = df['Churn'].value_counts().values, names = df['Churn'].value_counts().index, hole=0.4,
             color=df['Churn'].value_counts().index,
             color_discrete_sequence=color_list_default[:2])
fig.update_layout(height = 400,
                  width = 400,
                  showlegend = True,
                  title='Churn')
fig.update_traces(textposition = 'inside', textinfo = 'percent+label')
fig.show()

Most of the customers stay with the company.

# Data Preprocessing

**Find correlation**

In [ ]:
df_encode = df.copy() # Make duplicate dataframe
label_encoder = LabelEncoder()

# Encode the categorical columns
for col in df_encode.dtypes[(df_encode.dtypes=='object') | (df_encode.dtypes=='category')].index:
    df_encode[col] = label_encoder.fit_transform(df_encode[col])

# Find correlation between churn vs all columns
df_corr = df_encode.corr()
df_corr.loc[:,['Churn']].sort_values(by='Churn', ascending=False)

,Churn
Churn,1.000000
MonthlyCharges,0.194508
PaperlessBilling,0.190891
SeniorCitizen,0.151619
PaymentMethod,0.106238
MultipleLines,0.040175
PhoneService,0.011323
gender,-0.008763
StreamingTV,-0.034591
StreamingMovies,-0.036492


The 'MultipleLines', 'PhoneService', 'gender', 'StreamingTV', 'StreamingMovies', and 'InternetService' have low correlation values (below 0.01)

**Chi Square Test**

In [ ]:
X = df_encode.drop('Churn', axis=1)
y = df_encode['Churn']

X['tenure'] = MinMaxScaler().fit_transform(df_encode[['tenure']])
X['MonthlyCharges'] = MinMaxScaler().fit_transform(df_encode[['MonthlyCharges']])
X['TotalCharges'] = MinMaxScaler().fit_transform(df_encode[['TotalCharges']])

chi_scores = chi2(X,y)
p_val = pd.Series(chi_scores[1], index=X.columns).sort_values(ascending=False)

p_val.iloc[np.where(p_val >= 0.05)]

,0
PhoneService,0.767464
gender,0.605214


'PhoneService' and 'gender' have a high pvalue (more than 0.05). It indicates that these features have no significant association with the 'churn' label. So, in the next process we will not include these features into the models.

**Define the features and the label for the machine learning models**

In [ ]:
X = df.drop(['gender','TenureYear', 'PhoneService', 'Churn'], axis=1)
y = df['Churn']

**Encode the categorical columns**

In [ ]:
# Label encoding for 'Contract' column
map_contract = {'Month-to-month' : 0,
                'One year' : 1,
                'Two year' : 2}

X['Contract'] = X['Contract'].replace(map_contract)

In [ ]:
# One hot encoding for all feature columns except 'Contract' column
col_one_hot = X.drop(['Contract', 'tenure', 'MonthlyCharges', 'TotalCharges'], axis=1).columns

for col in col_one_hot:
    dummies = pd.get_dummies(X[col], drop_first=True)
    X = X.drop([col], axis=1)
    X = pd.concat([X, dummies], axis=1)

**Normalize 'tenure', 'MonthlyCharges', 'TotalCharges' columns**

Based on the exploratory data analysis process conducted previously, the data for 'tenure', 'MonthlyCharges', and 'TotalCharges' are not normally distributed, so we will perform normalization instead of standardization.

In [ ]:
# Before normalization
X[['tenure', 'MonthlyCharges', 'TotalCharges']].describe()

,tenure,MonthlyCharges,TotalCharges
count,7021.000000,7021.000000,7021.000000
mean,32.469449,64.851894,2286.765026
std,24.534965,30.069001,2266.855057
min,0.000000,18.250000,0.000000
25%,9.000000,35.750000,403.350000
50%,29.000000,70.400000,1400.550000
75%,55.000000,89.900000,3801.700000
max,72.000000,118.750000,8684.800000


In [ ]:
# Normalization
X['tenure'] = MinMaxScaler().fit_transform(X[['tenure']])
X['MonthlyCharges'] = MinMaxScaler().fit_transform(X[['MonthlyCharges']])
X['TotalCharges'] = MinMaxScaler().fit_transform(X[['TotalCharges']])

In [ ]:
# After normalization
X[['tenure', 'MonthlyCharges', 'TotalCharges']].describe()

,tenure,MonthlyCharges,TotalCharges
count,7021.000000,7021.000000,7021.000000
mean,0.450965,0.463700,0.263307
std,0.340763,0.299194,0.261014
min,0.000000,0.000000,0.000000
25%,0.125000,0.174129,0.046443
50%,0.402778,0.518905,0.161265
75%,0.763889,0.712935,0.437742
max,1.000000,1.000000,1.000000


**Split the data into training data and testing data with proportion 75:25**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=9, stratify=y)

In [ ]:
y_test.value_counts()

,count
Churn,
No,1292
Yes,464


In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

# Data Modelling

**Define some functions**

In [ ]:
# Function for Cross Validation
def cross_val(X, y, model, skf, is_xgb = False, threshold=0.5):
    cv = {'tn' : [], 'fp' : [], 'fn' : [], 'tp' : [],
          'prec_yes' : [], 'recall_yes' : [], 'prec_no' : [], 'recall_no' : [],
          'accuracy' : [], 'f1_score' : []}

    for train, val in skf.split(X, y):
        X_cv_train, X_cv_val = X[train], X[val]
        y_cv_train, y_cv_val = y.iloc[train], y.iloc[val]

        model.fit(X_cv_train, y_cv_train) # Training per fold

        if is_xgb:
            y_cv_pred = np.where(model.predict_proba(X_cv_val)[:,1] > threshold, 1, 0) # Validation per fold
        else:
            y_cv_pred = np.where(model.predict_proba(X_cv_val)[:,1] > threshold, 'Yes', 'No') # Validation per fold

        cm_ravel = confusion_matrix(y_cv_val, y_cv_pred).ravel()

        for i, cm_i in enumerate(['tn', 'fp', 'fn', 'tp']):
            cv[cm_i].append(cm_ravel[i])

        cr = classification_report(y_cv_val, y_cv_pred, output_dict=True)
        if not is_xgb:
            label = ['Yes', 'No']
        else:
            label = ['1', '0']

        cv['prec_yes'].append(cr[label[0]]['precision'])
        cv['recall_yes'].append(cr[label[0]]['recall'])
        cv['prec_no'].append(cr[label[1]]['precision'])
        cv['recall_no'].append(cr[label[1]]['recall'])
        cv['accuracy'].append(cr['accuracy'])
        cv['f1_score'].append(cr[label[0]]['f1-score'])

    df_cv = pd.DataFrame(cv)
    df_mean = pd.DataFrame([['']*4 + list(df_cv.mean().values[4:])], columns=df_cv.columns, index=['mean'])
    df_std_dev = pd.DataFrame([['']*4 + list(df_cv.std().values[4:])], columns=df_cv.columns, index=['std_dev'])

    return pd.concat([df_cv, df_mean, df_std_dev], axis=0)

In [ ]:
# Function to check model
def check_model(model, is_xgb = False, threshold=0.5):
    if is_xgb:
        model.fit(X_train, y_train.replace({'Yes' : 1, 'No' : 0}))
        label = np.array(['No', 'Yes'], dtype='object')
    else:
        model.fit(X_train, y_train)
        label = model.classes_
    y_predict = np.where(model.predict_proba(X_test)[:,1] > threshold, 'Yes', 'No')

    fig = px.imshow(pd.DataFrame(confusion_matrix(y_test, y_predict),
                                 columns = 'Predicted ' + label,
                                 index = 'Actual ' + label),
                    text_auto=True,
                    aspect='auto',
                    width = 600,
                    color_continuous_scale=px.colors.sequential.Emrld)

    fig.show()
    print(classification_report(y_test, y_predict))

**Logistic Regression vs XGBoost models**

In [ ]:
# Models
logreg = LogisticRegression()
xgb = XGBClassifier()

Cross Validation

In [ ]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=20)

# Logistic regression cross validation
cross_val(X_train, y_train, logreg, skf)

,tn,fp,fn,tp,prec_yes,recall_yes,prec_no,recall_no,accuracy,f1_score
0,883,85,164,185,0.685185,0.530086,0.843362,0.912190,0.810934,0.597738
1,865,103,166,182,0.638596,0.522989,0.838991,0.893595,0.795593,0.575039
2,875,93,142,206,0.688963,0.591954,0.860374,0.903926,0.821429,0.636785
3,859,109,161,187,0.631757,0.537356,0.842157,0.887397,0.794833,0.580745
mean,,,,,0.661125,0.545596,0.846221,0.899277,0.805697,0.597577
std_dev,,,,,0.030132,0.031457,0.009613,0.010982,0.012846,0.027860


In [ ]:
# XGBoost Cross Validation
cross_val(X_train, y_train.replace({'Yes' : 1, 'No' : 0}), xgb, skf, is_xgb=True)

,tn,fp,fn,tp,prec_yes,recall_yes,prec_no,recall_no,accuracy,f1_score
0,875,93,179,170,0.646388,0.487106,0.830171,0.903926,0.793470,0.555556
1,841,127,149,199,0.610429,0.571839,0.849495,0.868802,0.790274,0.590504
2,862,106,165,183,0.633218,0.525862,0.839338,0.890496,0.794073,0.574568
3,868,100,164,184,0.647887,0.528736,0.841085,0.896694,0.799392,0.582278
mean,,,,,0.634481,0.528386,0.840022,0.889979,0.794302,0.575727
std_dev,,,,,0.017336,0.034635,0.007924,0.015148,0.003781,0.014939


Both models perform quite consistent results for each fold of cross validation.

Check performace for default models

In [ ]:
# Logistic regression default model
check_model(logreg)

              precision    recall  f1-score   support

          No       0.84      0.88      0.86      1292
         Yes       0.61      0.53      0.57       464

    accuracy                           0.79      1756
   macro avg       0.72      0.70      0.71      1756
weighted avg       0.78      0.79      0.78      1756



In [ ]:
# XGBoost default model
check_model(xgb, is_xgb=True)

              precision    recall  f1-score   support

          No       0.83      0.87      0.85      1292
         Yes       0.57      0.50      0.54       464

    accuracy                           0.77      1756
   macro avg       0.70      0.68      0.69      1756
weighted avg       0.76      0.77      0.76      1756



Check performance for models with hyperparameter tuning

In [ ]:
# Logistic regression model with hyperparameter tuning
logreg = LogisticRegression(max_iter=1000, random_state=21, C=1, penalty=None)
check_model(logreg)

              precision    recall  f1-score   support

          No       0.84      0.87      0.86      1292
         Yes       0.61      0.55      0.58       464

    accuracy                           0.79      1756
   macro avg       0.73      0.71      0.72      1756
weighted avg       0.78      0.79      0.78      1756



In [ ]:
# XGBoost model with hyperparameter tuning
xgb = XGBClassifier(max_depth=1, alpha=0.01, gamma=0.1, n_estimators=100, random_state=21, seed=4)
check_model(xgb, is_xgb=True)

              precision    recall  f1-score   support

          No       0.84      0.88      0.86      1292
         Yes       0.61      0.52      0.56       464

    accuracy                           0.79      1756
   macro avg       0.73      0.70      0.71      1756
weighted avg       0.78      0.79      0.78      1756



Check performance for models with threshold modification

In [ ]:
# Logistic regression model with threshold modification
check_model(logreg, threshold=0.45)

              precision    recall  f1-score   support

          No       0.86      0.85      0.86      1292
         Yes       0.60      0.61      0.61       464

    accuracy                           0.79      1756
   macro avg       0.73      0.73      0.73      1756
weighted avg       0.79      0.79      0.79      1756



In [ ]:
# XGBoost model with threshold modification
check_model(xgb, is_xgb=True, threshold=0.45)

              precision    recall  f1-score   support

          No       0.85      0.86      0.86      1292
         Yes       0.60      0.59      0.60       464

    accuracy                           0.79      1756
   macro avg       0.73      0.73      0.73      1756
weighted avg       0.79      0.79      0.79      1756



Here, we only focus on the customers who actually churn because we want to retain them. So, we will use the precision 'yes' value to compare the performance of the models.

Based on the classification reports, the logistic regression model with hyperparameter tuning and threshold modification results precision 'yes' of 0.61. This value is the highest precision 'yes' that models can get from this scenario. The model can correctly predict 61% of all customers who actually churn.

# Conclusion

In conclusion, customers vulnerable to churn are customers who ...
1. Have subscribed for less than a year.
2. Have month-to-month subscription terms.
3. Don’t have any security services (OnlineSecurity, OnlineBackup, DeviceProtection, and TechSupport).

Several ways we can do to retain those vulnerable customers include:

1. Focusing on customers who likely to churn.
2. Offering them various service packages in smaller quantities at affordable prices. They might churn if they don't find suitable options that match their financial situation.
3. Providing monthly bonuses, such as discounted security services every month or at the end of their subscription period.